<a href="https://colab.research.google.com/github/dtuleva/Recipe_Recommendation_System/blob/main/tfrd_basic_ranking_recreate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders

In [2]:
!pip install mlflow --quiet

In [3]:
# import os
# import pprint
# import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_recommenders as tfrs

import mlflow

# Recreate Recommending movies: ranking

[tutorial link](https://www.tensorflow.org/recommenders/examples/basic_ranking)

## Preparing the dataset

We're going to use the same data as the [retrieval](basic_retrieval) tutorial. This time, we're also going to keep the ratings: these are the objectives we are trying to predict.

As before, we'll split the data by putting 80% of the ratings in the train set, and 20% in the test set.



In [4]:
ratings_data = pd.read_csv("/content/mock-data_interaction.csv", index_col = 0)
# recipes_data = pd.read_csv("/content/mock-data_recipe.csv")

In [5]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_data))
# recipes = tf.data.Dataset.from_tensor_slices(dict(recipes_data))

In [6]:
for rating in ratings.take(1):
  for item in rating.items():
    print(item)

('user_id', <tf.Tensor: shape=(), dtype=int64, numpy=8542392>)
('recipe_id', <tf.Tensor: shape=(), dtype=int64, numpy=222388>)
('rating', <tf.Tensor: shape=(), dtype=int64, numpy=5>)
('dateLastModified', <tf.Tensor: shape=(), dtype=string, numpy=b'2017-04-22T12:46:43.663\n'>)


In [7]:
# feature selection
ratings = ratings.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"],
    "rating": x["rating"]
})

In [8]:
for rating in ratings.take(1):
  for item in rating.items():
    print(item)

('recipe_id', <tf.Tensor: shape=(), dtype=int64, numpy=222388>)
('user_id', <tf.Tensor: shape=(), dtype=int64, numpy=8542392>)
('rating', <tf.Tensor: shape=(), dtype=int64, numpy=5>)


In [9]:
test_recipe_id = 222388
test_user_id = 8542392

In [10]:
recipes = ratings.map(lambda x: x["recipe_id"])

In [11]:
ratings_cardinality = ratings.cardinality()
ratings_cardinality

<tf.Tensor: shape=(), dtype=int64, numpy=8671>

In [12]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(ratings.cardinality(), seed=42, reshuffle_each_iteration=False)

train = shuffled.take(6_000)
test = shuffled.skip(6_000).take(2_000)

Let's also figure out unique user ids and movie titles present in the data.

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [13]:
unique_user_ids = ratings_data.user_id.unique()
len(unique_user_ids)

8266

In [14]:
unique_recipe_ids = ratings_data.recipe_id.unique()
len(unique_recipe_ids)

100

In [15]:
unique_user_ids = np.unique(list(unique_user_ids))
unique_recipe_ids = np.unique(list(unique_recipe_ids))

## Implementing a model

### Architecture

Ranking models do not face the same efficiency constraints as retrieval models do, and so we have a little bit more freedom in our choice of architectures.

A model composed of multiple stacked dense layers is a relatively common architecture for ranking tasks. We can implement it as follows:

In [16]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.IntegerLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.IntegerLookup(
        vocabulary=unique_recipe_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

This model takes user ids and movie titles, and outputs a predicted rating:

In [17]:
RankingModel()(([test_user_id], [test_recipe_id])) # not trained, just cheching

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.01076964]], dtype=float32)>

### Loss and metrics

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Ranking` task object: a convenience wrapper that bundles together the loss function and metric computation.

We'll use it together with the `MeanSquaredError` Keras loss in order to predict the ratings.

In [18]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

The task itself is a Keras layer that takes true and predicted as arguments, and returns the computed loss. We'll use that to implement the model's training loop.

### The full model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines bulding models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [27]:
class RecipeRankingModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["recipe_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [28]:
model = RecipeRankingModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [29]:
cached_train = train.shuffle(6000).batch(512).cache()
cached_test = test.batch(2000).cache()

Then train the  model:

In [30]:
# for el in cached_train.take(1):
#   print(type(el))
#   el.pop("rating")
#   print(el)

In [31]:
logs = "/content/log"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs)

In [24]:
experiment = mlflow.create_experiment("tfrs_basic_ranking_recreate_01")

In [32]:
mlflow.autolog()


2024/01/10 13:23:28 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


In [33]:
with mlflow.start_run(experiment_id = experiment, run_name="autolog_01") as run:
    model.fit(cached_train, epochs = 10, callbacks = [tensorboard_callback])

2024/01/10 13:23:30 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: The layer "recipe_ranking_model_1" has never been called and thus has no defined input shape. Note that the `input_shape` property is only available for Functional and Sequential models.


Epoch 1/10
{'recipe_id': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>, 'user_id': <tf.Tensor 'IteratorGetNext:2' shape=(None,) dtype=int64>, 'rating': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=int64>}
{'recipe_id': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>, 'user_id': <tf.Tensor 'IteratorGetNext:2' shape=(None,) dtype=int64>, 'rating': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=int64>}
12/12 [==============================] - 1s 8ms/step - root_mean_squared_error: 2.7605 - loss: 6.9516 - regularization_loss: 0.0000e+00 - total_loss: 6.9516
Epoch 2/10
12/12 [==============================] - 0s 8ms/step - root_mean_squared_error: 0.8126 - loss: 0.6606 - regularization_loss: 0.0000e+00 - total_loss: 0.6606
Epoch 3/10
12/12 [==============================] - 0s 7ms/step - root_mean_squared_error: 0.7901 - loss: 0.6271 - regularization_loss: 0.0000e+00 - total_loss: 0.6271
Epoch 4/10
12/12 [==============================] - 0s 8ms/step - root_m

2024/01/10 13:23:33 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'tensorflow.python.data.ops.cache_op.CacheDataset'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)
2024/01/10 13:23:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/01/10 13:23:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


As the model trains, the loss is falling and the RMSE metric is improving.

Finally, we can evaluate our model on the test set:

In [34]:
model.evaluate(cached_test, return_dict = True, callbacks = [tensorboard_callback])

{'recipe_id': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>, 'user_id': <tf.Tensor 'IteratorGetNext:2' shape=(None,) dtype=int64>, 'rating': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=int64>}
1/1 [==============================] - 0s 446ms/step - root_mean_squared_error: 0.8285 - loss: 0.6864 - regularization_loss: 0.0000e+00 - total_loss: 0.6864


{'root_mean_squared_error': 0.8284972310066223,
 'loss': 0.6864076852798462,
 'regularization_loss': 0,
 'total_loss': 0.6864076852798462}

The lower the RMSE metric, the more accurate our model is at predicting ratings.

## Testing the ranking model

Now we can test the ranking model by computing predictions for a set of movies and then rank these movies based on the predictions:

In [36]:
test_ratings = {}
test_movie_titles = [test_recipe_id]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array([test_recipe_id]),
      "recipe_id": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
222388: [[4.348587]]


In [37]:
# real rating 5; TODO: add more test pairs

## Exporting for serving

The model can be easily exported for serving:


In [38]:
tf.saved_model.save(model, "tfrs_basic_ranking_model")

We can now load it back and perform predictions:

In [39]:
loaded = tf.saved_model.load("/content/tfrs_basic_ranking_model")

loaded({"user_id": np.array([test_user_id]), "recipe_id": [test_recipe_id]}).numpy()

array([[4.3248014]], dtype=float32)

## Convert the model to TensorFLow Lite

nope

## Next steps

The model above gives us a decent start towards building a ranking system.

Of course, making a practical ranking system requires much more effort.

In most cases, a ranking model can be substantially improved by using more features rather than just user and candidate identifiers. To see how to do that, have a look at the [side features](featurization) tutorial.

A careful understanding of the objectives worth optimizing is also necessary. To get started on building a recommender that optimizes multiple objectives, have a look at our [multitask](multitask) tutorial.

## Zip saved model, tensorboard and mlflow runs for downloading from colab

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

# %reload_ext tensorboard

%tensorboard --logdir="/content/log"

In [40]:
# zip mlflow logs
!zip -r /content/mlflow_basic_ranking.zip /content/mlruns

  adding: content/mlruns/ (stored 0%)
  adding: content/mlruns/0/ (stored 0%)
  adding: content/mlruns/0/datasets/ (stored 0%)
  adding: content/mlruns/0/datasets/eb231e126f93da5e54765425072f5747/ (stored 0%)
  adding: content/mlruns/0/datasets/eb231e126f93da5e54765425072f5747/meta.yaml (deflated 53%)
  adding: content/mlruns/0/datasets/74ea8d26a002b2df1f7bda269c6014c4/ (stored 0%)
  adding: content/mlruns/0/datasets/74ea8d26a002b2df1f7bda269c6014c4/meta.yaml (deflated 52%)
  adding: content/mlruns/0/f8d093dd2ea1433082acc13792e79464/ (stored 0%)
  adding: content/mlruns/0/f8d093dd2ea1433082acc13792e79464/metrics/ (stored 0%)
  adding: content/mlruns/0/f8d093dd2ea1433082acc13792e79464/tags/ (stored 0%)
  adding: content/mlruns/0/f8d093dd2ea1433082acc13792e79464/tags/mlflow.user (stored 0%)
  adding: content/mlruns/0/f8d093dd2ea1433082acc13792e79464/tags/mlflow.source.name (deflated 5%)
  adding: content/mlruns/0/f8d093dd2ea1433082acc13792e79464/tags/mlflow.runName (stored 0%)
  adding: 

In [41]:
# zip model
!zip -r /content/tfrs_basic_ranking_model /content/tfrs_basic_ranking_model

  adding: content/tfrs_basic_ranking_model/ (stored 0%)
  adding: content/tfrs_basic_ranking_model/fingerprint.pb (stored 0%)
  adding: content/tfrs_basic_ranking_model/assets/ (stored 0%)
  adding: content/tfrs_basic_ranking_model/variables/ (stored 0%)
  adding: content/tfrs_basic_ranking_model/variables/variables.data-00000-of-00001 (deflated 33%)
  adding: content/tfrs_basic_ranking_model/variables/variables.index (deflated 56%)
  adding: content/tfrs_basic_ranking_model/saved_model.pb (deflated 81%)


In [42]:
# zip tensorboard logs
!zip -r /content/log.zip /content/log/

  adding: content/log/ (stored 0%)
  adding: content/log/validation/ (stored 0%)
  adding: content/log/validation/events.out.tfevents.1704893179.c4d4cdb7b2eb.15866.3.v2 (deflated 41%)
  adding: content/log/train/ (stored 0%)
  adding: content/log/train/events.out.tfevents.1704893010.c4d4cdb7b2eb.15866.1.v2 (deflated 88%)
  adding: content/log/train/events.out.tfevents.1704892392.c4d4cdb7b2eb.239.3.v2 (deflated 8%)
  adding: content/log/train/events.out.tfevents.1704892305.c4d4cdb7b2eb.239.2.v2 (deflated 5%)
  adding: content/log/train/events.out.tfevents.1704891361.c4d4cdb7b2eb.239.0.v2 (deflated 5%)
  adding: content/log/train/events.out.tfevents.1704893179.c4d4cdb7b2eb.15866.2.v2 (deflated 5%)
  adding: content/log/train/events.out.tfevents.1704892431.c4d4cdb7b2eb.239.5.v2 (deflated 5%)
  adding: content/log/train/events.out.tfevents.1704892512.c4d4cdb7b2eb.13851.0.v2 (deflated 6%)
  adding: content/log/train/events.out.tfevents.1704892961.c4d4cdb7b2eb.15866.0.v2 (deflated 5%)
  addi